## 28_combine_census_data_extended
## This notebook handles adding extended census-tract data to srprec, then adding vote data

In [1]:
# imports
import pandas as pd
import numpy as np
import requests
import pickle as pkl
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

### Converter file

In [3]:
#bring in converter file, created in 25_srprec_tract_converter page
convert18 = pd.read_pickle('./data/df_convert18.pkl')

In [4]:
convert18.head(3)

,subindex,srprec_orig,tract,pctsrprec_tract
0,0,7701,110304,100.000000
1,0,75702,75809,97.095134
2,1,75702,75814,2.904866


In [5]:
convert18.shape

(2011, 4)

In [6]:
convert18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2011 entries, 0 to 2010
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   subindex         2011 non-null   int64  
 1   srprec_orig      2011 non-null   int64  
 2   tract            2011 non-null   int64  
 3   pctsrprec_tract  2011 non-null   float64
dtypes: float64(1), int64(3)
memory usage: 63.0 KB


### Merge the full dataframes
#### Steps:
1. Attach census data on 'tract', apply weights by 'tract', aggregate to 'srprec'.
2. Attach vote data on 'srprec'

#### Step 1:  Attach census data on 'tract', apply weights by 'tract', aggregate to 'srprec'.

#### Census data set

In [7]:
census18_DP03 = pd.read_pickle('./census_data/DP03_extended.pkl')
census18_DP03.shape

(583, 18)

In [8]:
census18_DP03.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 583 entries, 1 to 583
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   GEO_ID                583 non-null    object
 1   tract                 583 non-null    int64 
 2   employed              583 non-null    int64 
 3   unemployed            583 non-null    int64 
 4   empl_military         583 non-null    int64 
 5   not_inlaborforce      583 non-null    int64 
 6   working_women         583 non-null    int64 
 7   parents_work_under6   583 non-null    int64 
 8   parents_work_0617     583 non-null    int64 
 9   occ_mgmt_sci_art      583 non-null    int64 
 10  occ_service_sector    583 non-null    int64 
 11  occ_sales_gen_office  583 non-null    int64 
 12  occ_constr_maintc     583 non-null    int64 
 13  occ_manuf_transpo     583 non-null    int64 
 14  hh_med_income         583 non-null    int64 
 15  hlthins_priv          583 non-null    in

In [9]:
census18_DP03.drop(['GEO_ID'], axis=1, inplace=True)

In [10]:

census18_DP03.head(3)

,tract,employed,unemployed,empl_military,not_inlaborforce,working_women,parents_work_under6,parents_work_0617,occ_mgmt_sci_art,occ_service_sector,occ_sales_gen_office,occ_constr_maintc,occ_manuf_transpo,hh_med_income,hlthins_priv,hlthins_public,hlthins_none
1,62644,3811,156,0,2800,1652,133,862,2547,262,819,91,92,146953,7854,1504,245
2,62640,2320,116,10,616,1234,122,447,1202,287,563,63,205,84632,2687,889,263
3,63008,412,19,0,402,239,4,43,255,22,104,0,31,100396,668,396,0


In [11]:
census18_DP03.shape

(583, 17)

#### Merge census data into convert18 df on 'tract' to create combo df

In [12]:
# pd.merge(left=c, right=b, how='outer', left_on=['tract'], right_on=['tract'], indicator=True, suffixes=('_c', '_b'))
combo = pd.merge(left=convert18, right=census18_DP03, how='outer', left_on=['tract'], right_on=['tract'], indicator=True)

In [13]:
combo.head(10)

,subindex,srprec_orig,tract,pctsrprec_tract,employed,unemployed,empl_military,not_inlaborforce,working_women,parents_work_under6,...,occ_mgmt_sci_art,occ_service_sector,occ_sales_gen_office,occ_constr_maintc,occ_manuf_transpo,hh_med_income,hlthins_priv,hlthins_public,hlthins_none,_merge
0,0.0,7701.0,110304,100.000000,2614,187,0,1442,1305,244,...,875,621,697,195,226,105746,3707,1497,181,both
1,0.0,7140.0,110304,100.000000,2614,187,0,1442,1305,244,...,875,621,697,195,226,105746,3707,1497,181,both
2,0.0,7138.0,110304,100.000000,2614,187,0,1442,1305,244,...,875,621,697,195,226,105746,3707,1497,181,both
3,0.0,11075.0,110304,100.000000,2614,187,0,1442,1305,244,...,875,621,697,195,226,105746,3707,1497,181,both
4,2.0,11043.0,110304,25.630594,2614,187,0,1442,1305,244,...,875,621,697,195,226,105746,3707,1497,181,both
5,0.0,75702.0,75809,97.095134,1188,107,0,1191,548,10,...,725,133,238,71,21,165278,2281,1012,71,both
6,0.0,75116.0,75809,70.290635,1188,107,0,1191,548,10,...,725,133,238,71,21,165278,2281,1012,71,both
7,0.0,75110.0,75809,16.946309,1188,107,0,1191,548,10,...,725,133,238,71,21,165278,2281,1012,71,both
8,0.0,63710.0,75809,4.524089,1188,107,0,1191,548,10,...,725,133,238,71,21,165278,2281,1012,71,both
9,1.0,75702.0,75814,2.904866,1567,114,0,716,790,111,...,890,199,388,0,90,130438,2500,566,68,both


In [14]:
combo.shape

(2012, 21)

In [15]:
combo['_merge'].value_counts()

both          2011
right_only       1
left_only        0
Name: _merge, dtype: int64

In [16]:
combo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2012 entries, 0 to 2011
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   subindex              2011 non-null   float64 
 1   srprec_orig           2011 non-null   float64 
 2   tract                 2012 non-null   int64   
 3   pctsrprec_tract       2011 non-null   float64 
 4   employed              2012 non-null   int64   
 5   unemployed            2012 non-null   int64   
 6   empl_military         2012 non-null   int64   
 7   not_inlaborforce      2012 non-null   int64   
 8   working_women         2012 non-null   int64   
 9   parents_work_under6   2012 non-null   int64   
 10  parents_work_0617     2012 non-null   int64   
 11  occ_mgmt_sci_art      2012 non-null   int64   
 12  occ_service_sector    2012 non-null   int64   
 13  occ_sales_gen_office  2012 non-null   int64   
 14  occ_constr_maintc     2012 non-null   int64   
 15  occ_

#### Inspect resulting dataframe:  'srprec' with multiple tracts, nulls

In [17]:
#stackoverflow:  https://stackoverflow.com/questions/14247586/how-to-select-rows-with-one-or-more-nulls-from-a-pandas-dataframe-without-listin
combo[pd.isnull(combo).any(axis=1)]

,subindex,srprec_orig,tract,pctsrprec_tract,employed,unemployed,empl_military,not_inlaborforce,working_women,parents_work_under6,...,occ_mgmt_sci_art,occ_service_sector,occ_sales_gen_office,occ_constr_maintc,occ_manuf_transpo,hh_med_income,hlthins_priv,hlthins_public,hlthins_none,_merge
2011,NaN,NaN,990100,NaN,0,0,0,0,0,0,...,0,0,0,0,0,-666666666,0,0,0,right_only


In [18]:
combo.loc[combo['srprec_orig'] == 14082]

,subindex,srprec_orig,tract,pctsrprec_tract,employed,unemployed,empl_military,not_inlaborforce,working_women,parents_work_under6,...,occ_mgmt_sci_art,occ_service_sector,occ_sales_gen_office,occ_constr_maintc,occ_manuf_transpo,hh_med_income,hlthins_priv,hlthins_public,hlthins_none,_merge
1567,0.0,14082.0,88101,0.127389,1123,55,7,552,425,138,...,356,275,306,98,88,74665,1368,982,186,both
1878,1.0,14082.0,110003,99.872611,1496,47,14,930,763,202,...,731,116,372,150,127,86176,2521,954,94,both


In [19]:
combo.loc[combo['_merge'] == 'right_only']

,subindex,srprec_orig,tract,pctsrprec_tract,employed,unemployed,empl_military,not_inlaborforce,working_women,parents_work_under6,...,occ_mgmt_sci_art,occ_service_sector,occ_sales_gen_office,occ_constr_maintc,occ_manuf_transpo,hh_med_income,hlthins_priv,hlthins_public,hlthins_none,_merge
2011,NaN,NaN,990100,NaN,0,0,0,0,0,0,...,0,0,0,0,0,-666666666,0,0,0,right_only


In [20]:
combo.drop(index=2011, inplace=True)

In [21]:
combo['pctsrprec_tract'] = round(combo['pctsrprec_tract'], 2)

In [38]:
combo.head(10)

,subindex,srprec_orig,tract,pctsrprec_tract,employed,unemployed,empl_military,not_inlaborforce,working_women,parents_work_under6,...,occ_mgmt_sci_art,occ_service_sector,occ_sales_gen_office,occ_constr_maintc,occ_manuf_transpo,hh_med_income,hlthins_priv,hlthins_public,hlthins_none,_merge
0,0.0,7701.0,110304,100.00,2614,187,0,1442,1305,244,...,875,621,697,195,226,105746,3707,1497,181,both
1,0.0,7140.0,110304,100.00,2614,187,0,1442,1305,244,...,875,621,697,195,226,105746,3707,1497,181,both
2,0.0,7138.0,110304,100.00,2614,187,0,1442,1305,244,...,875,621,697,195,226,105746,3707,1497,181,both
3,0.0,11075.0,110304,100.00,2614,187,0,1442,1305,244,...,875,621,697,195,226,105746,3707,1497,181,both
4,2.0,11043.0,110304,25.63,2614,187,0,1442,1305,244,...,875,621,697,195,226,105746,3707,1497,181,both
5,0.0,75702.0,75809,97.10,1188,107,0,1191,548,10,...,725,133,238,71,21,165278,2281,1012,71,both
6,0.0,75116.0,75809,70.29,1188,107,0,1191,548,10,...,725,133,238,71,21,165278,2281,1012,71,both
7,0.0,75110.0,75809,16.95,1188,107,0,1191,548,10,...,725,133,238,71,21,165278,2281,1012,71,both
8,0.0,63710.0,75809,4.52,1188,107,0,1191,548,10,...,725,133,238,71,21,165278,2281,1012,71,both
9,1.0,75702.0,75814,2.90,1567,114,0,716,790,111,...,890,199,388,0,90,130438,2500,566,68,both


#### Calculate weighted values for each tract

**NOTE** Opportunity to perform this calculation as a for-loop or function, but for time-constraints and precision, I calculated by hand

In [47]:
combo['employed_wgt'] = round((combo['pctsrprec_tract']/100 * combo['employed']), 0)

In [49]:
combo['unemployed_wgt'] = round((combo['pctsrprec_tract']/100 * combo['unemployed']), 0)
combo['empl_military_wgt'] = round((combo['pctsrprec_tract']/100 * combo['empl_military']), 0)
combo['not_inlaborforce_wgt'] = round((combo['pctsrprec_tract']/100 * combo['not_inlaborforce']), 0)

In [51]:
combo['working_women_wgt'] = round((combo['pctsrprec_tract']/100 * combo['working_women']), 0)
combo['parents_work_under6_wgt'] = round((combo['pctsrprec_tract']/100 * combo['parents_work_under6']), 0)
combo['parents_work_0617_wgt'] = round((combo['pctsrprec_tract']/100 * combo['parents_work_0617']), 0)

In [53]:
combo['occ_mgmt_sci_art_wgt'] = round((combo['pctsrprec_tract']/100 * combo['occ_mgmt_sci_art']), 0)
combo['occ_service_sector_wgt'] = round((combo['pctsrprec_tract']/100 * combo['occ_service_sector']), 0)
combo['occ_sales_gen_office_wgt'] = round((combo['pctsrprec_tract']/100 * combo['occ_sales_gen_office']), 0)

In [55]:
combo['occ_constr_maintc_wgt'] = round((combo['pctsrprec_tract']/100 * combo['occ_constr_maintc']), 0)
combo['occ_manuf_transpo_wgt'] = round((combo['pctsrprec_tract']/100 * combo['occ_manuf_transpo']), 0)
combo['hh_med_income_wgt'] = round((combo['pctsrprec_tract']/100 * combo['hh_med_income']), 0)

In [57]:
combo['hlthins_priv_wgt'] = round((combo['pctsrprec_tract']/100 * combo['hlthins_priv']), 0)
combo['hlthins_public_wgt'] = round((combo['pctsrprec_tract']/100 * combo['hlthins_public']), 0)
combo['hlthins_none_wgt'] = round((combo['pctsrprec_tract']/100 * combo['hlthins_none']), 0)

In [58]:
combo.head(10).T

,0,1,2,3,4,5,6,7,8,9
subindex,0,0,0,0,2,0,0,0,0,1
srprec_orig,7701,7140,7138,11075,11043,75702,75116,75110,63710,75702
tract,110304,110304,110304,110304,110304,75809,75809,75809,75809,75814
pctsrprec_tract,100,100,100,100,25.63,97.1,70.29,16.95,4.52,2.9
employed,2614,2614,2614,2614,2614,1188,1188,1188,1188,1567
unemployed,187,187,187,187,187,107,107,107,107,114
empl_military,0,0,0,0,0,0,0,0,0,0
not_inlaborforce,1442,1442,1442,1442,1442,1191,1191,1191,1191,716
working_women,1305,1305,1305,1305,1305,548,548,548,548,790
parents_work_under6,244,244,244,244,244,10,10,10,10,111


#### spot-checks

In [59]:
combo.loc[combo['srprec_orig'] == 75116]

,subindex,srprec_orig,tract,pctsrprec_tract,employed,unemployed,empl_military,not_inlaborforce,working_women,parents_work_under6,...,parents_work_0617_wgt,occ_mgmt_sci_art_wgt,occ_service_sector_wgt,occ_sales_gen_office_wgt,occ_constr_maintc_wgt,occ_manuf_transpo_wgt,hh_med_income_wgt,hlthins_priv_wgt,hlthins_public_wgt,hlthins_none_wgt
6,0.0,75116.0,75809,70.29,1188,107,0,1191,548,10,...,163.0,510.0,93.0,167.0,50.0,15.0,116174.0,1603.0,711.0,50.0
10,4.0,75116.0,75814,0.43,1567,114,0,716,790,111,...,1.0,4.0,1.0,2.0,0.0,0.0,561.0,11.0,2.0,0.0
18,1.0,75116.0,75810,21.10,1226,34,0,1361,505,48,...,55.0,130.0,19.0,82.0,18.0,10.0,25782.0,510.0,210.0,26.0
23,2.0,75116.0,75811,6.24,1576,89,0,906,571,76,...,16.0,32.0,22.0,18.0,13.0,13.0,4455.0,116.0,65.0,40.0
26,3.0,75116.0,75813,1.94,2490,152,0,1592,1188,132,...,5.0,22.0,8.0,13.0,2.0,3.0,2360.0,78.0,28.0,3.0


#### Aggregate results to 'srprec'

In [60]:
srprec_agg = combo.groupby('srprec_orig').sum()
srprec_agg

,subindex,tract,pctsrprec_tract,employed,unemployed,empl_military,not_inlaborforce,working_women,parents_work_under6,parents_work_0617,...,parents_work_0617_wgt,occ_mgmt_sci_art_wgt,occ_service_sector_wgt,occ_sales_gen_office_wgt,occ_constr_maintc_wgt,occ_manuf_transpo_wgt,hh_med_income_wgt,hlthins_priv_wgt,hlthins_public_wgt,hlthins_none_wgt
srprec_orig,,,,,,,,,,,,,,,,,,,,,
2001.0,3.0,184605,100.0,7357,444,0,4031,3316,781,2146,...,783.0,244.0,637.0,706.0,405.0,653.0,54480.0,2156.0,2846.0,1050.0
2002.0,0.0,11720,100.0,2881,217,9,1525,1030,257,1082,...,1082.0,363.0,573.0,383.0,569.0,993.0,44136.0,1885.0,3583.0,1700.0
2008.0,0.0,86702,100.0,3948,278,0,1438,1656,561,677,...,677.0,675.0,796.0,1064.0,368.0,1045.0,66970.0,3947.0,2926.0,1356.0
2009.0,3.0,284005,100.0,9352,464,0,7497,4413,830,2251,...,717.0,693.0,701.0,912.0,274.0,465.0,54967.0,3794.0,2708.0,822.0
2011.0,0.0,21905,100.0,2991,74,0,1401,1486,334,492,...,492.0,1364.0,331.0,1011.0,151.0,134.0,119595.0,4400.0,1523.0,259.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75116.0,10.0,379057,100.0,8047,496,0,5766,3602,377,1221,...,240.0,698.0,143.0,282.0,83.0,41.0,149332.0,2318.0,1016.0,119.0
75117.0,0.0,75810,100.0,1226,34,0,1361,505,48,262,...,262.0,615.0,91.0,387.0,87.0,46.0,122188.0,2418.0,997.0,125.0
75122.0,0.0,75810,100.0,1226,34,0,1361,505,48,262,...,262.0,615.0,91.0,387.0,87.0,46.0,122188.0,2418.0,997.0,125.0


In [61]:
srprec_agg.reset_index(inplace=True)

In [62]:
srprec_agg.head()

,srprec_orig,subindex,tract,pctsrprec_tract,employed,unemployed,empl_military,not_inlaborforce,working_women,parents_work_under6,...,parents_work_0617_wgt,occ_mgmt_sci_art_wgt,occ_service_sector_wgt,occ_sales_gen_office_wgt,occ_constr_maintc_wgt,occ_manuf_transpo_wgt,hh_med_income_wgt,hlthins_priv_wgt,hlthins_public_wgt,hlthins_none_wgt
0,2001.0,3.0,184605,100.0,7357,444,0,4031,3316,781,...,783.0,244.0,637.0,706.0,405.0,653.0,54480.0,2156.0,2846.0,1050.0
1,2002.0,0.0,11720,100.0,2881,217,9,1525,1030,257,...,1082.0,363.0,573.0,383.0,569.0,993.0,44136.0,1885.0,3583.0,1700.0
2,2008.0,0.0,86702,100.0,3948,278,0,1438,1656,561,...,677.0,675.0,796.0,1064.0,368.0,1045.0,66970.0,3947.0,2926.0,1356.0
3,2009.0,3.0,284005,100.0,9352,464,0,7497,4413,830,...,717.0,693.0,701.0,912.0,274.0,465.0,54967.0,3794.0,2708.0,822.0
4,2011.0,0.0,21905,100.0,2991,74,0,1401,1486,334,...,492.0,1364.0,331.0,1011.0,151.0,134.0,119595.0,4400.0,1523.0,259.0


#### Inspect

In [63]:
srprec_agg.loc[srprec_agg['srprec_orig'] == 75116]

,srprec_orig,subindex,tract,pctsrprec_tract,employed,unemployed,empl_military,not_inlaborforce,working_women,parents_work_under6,...,parents_work_0617_wgt,occ_mgmt_sci_art_wgt,occ_service_sector_wgt,occ_sales_gen_office_wgt,occ_constr_maintc_wgt,occ_manuf_transpo_wgt,hh_med_income_wgt,hlthins_priv_wgt,hlthins_public_wgt,hlthins_none_wgt
1329,75116.0,10.0,379057,100.0,8047,496,0,5766,3602,377,...,240.0,698.0,143.0,282.0,83.0,41.0,149332.0,2318.0,1016.0,119.0


#### **DROP TRACT DETAIL**

In [66]:
# cols that don't make sense in aggregate:
# subindex -- keep.  even if aggregated, it is a helpful indicator of an srprec which contains multiple tracts
# tract -- drop.  the aggregation adds tract numbers together, and it loses meaning
# original tract-level data -- drop.  no longer relevant, because it's been transformed to srprec-level

In [65]:
srprec_agg.columns.tolist() 

['srprec_orig',
 'subindex',
 'tract',
 'pctsrprec_tract',
 'employed',
 'unemployed',
 'empl_military',
 'not_inlaborforce',
 'working_women',
 'parents_work_under6',
 'parents_work_0617',
 'occ_mgmt_sci_art',
 'occ_service_sector',
 'occ_sales_gen_office',
 'occ_constr_maintc',
 'occ_manuf_transpo',
 'hh_med_income',
 'hlthins_priv',
 'hlthins_public',
 'hlthins_none',
 'employed_wgt',
 'unemployed_wgt',
 'empl_military_wgt',
 'not_inlaborforce_wgt',
 'working_women_wgt',
 'parents_work_under6_wgt',
 'parents_work_0617_wgt',
 'occ_mgmt_sci_art_wgt',
 'occ_service_sector_wgt',
 'occ_sales_gen_office_wgt',
 'occ_constr_maintc_wgt',
 'occ_manuf_transpo_wgt',
 'hh_med_income_wgt',
 'hlthins_priv_wgt',
 'hlthins_public_wgt',
 'hlthins_none_wgt']

In [67]:
srprec_census = srprec_agg[['srprec_orig',
 'subindex',
 'pctsrprec_tract',
 'employed_wgt',
 'unemployed_wgt',
 'empl_military_wgt',
 'not_inlaborforce_wgt',
 'working_women_wgt',
 'parents_work_under6_wgt',
 'parents_work_0617_wgt',
 'occ_mgmt_sci_art_wgt',
 'occ_service_sector_wgt',
 'occ_sales_gen_office_wgt',
 'occ_constr_maintc_wgt',
 'occ_manuf_transpo_wgt',
 'hh_med_income_wgt',
 'hlthins_priv_wgt',
 'hlthins_public_wgt',
 'hlthins_none_wgt']]
srprec_census

,srprec_orig,subindex,pctsrprec_tract,employed_wgt,unemployed_wgt,empl_military_wgt,not_inlaborforce_wgt,working_women_wgt,parents_work_under6_wgt,parents_work_0617_wgt,occ_mgmt_sci_art_wgt,occ_service_sector_wgt,occ_sales_gen_office_wgt,occ_constr_maintc_wgt,occ_manuf_transpo_wgt,hh_med_income_wgt,hlthins_priv_wgt,hlthins_public_wgt,hlthins_none_wgt
0,2001.0,3.0,100.0,2647.0,161.0,0.0,1372.0,1152.0,268.0,783.0,244.0,637.0,706.0,405.0,653.0,54480.0,2156.0,2846.0,1050.0
1,2002.0,0.0,100.0,2881.0,217.0,9.0,1525.0,1030.0,257.0,1082.0,363.0,573.0,383.0,569.0,993.0,44136.0,1885.0,3583.0,1700.0
2,2008.0,0.0,100.0,3948.0,278.0,0.0,1438.0,1656.0,561.0,677.0,675.0,796.0,1064.0,368.0,1045.0,66970.0,3947.0,2926.0,1356.0
3,2009.0,3.0,100.0,3045.0,147.0,0.0,2459.0,1447.0,273.0,717.0,693.0,701.0,912.0,274.0,465.0,54967.0,3794.0,2708.0,822.0
4,2011.0,0.0,100.0,2991.0,74.0,0.0,1401.0,1486.0,334.0,492.0,1364.0,331.0,1011.0,151.0,134.0,119595.0,4400.0,1523.0,259.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,75116.0,10.0,100.0,1247.0,91.0,0.0,1215.0,554.0,25.0,240.0,698.0,143.0,282.0,83.0,41.0,149332.0,2318.0,1016.0,119.0
1330,75117.0,0.0,100.0,1226.0,34.0,0.0,1361.0,505.0,48.0,262.0,615.0,91.0,387.0,87.0,46.0,122188.0,2418.0,997.0,125.0
1331,75122.0,0.0,100.0,1226.0,34.0,0.0,1361.0,505.0,48.0,262.0,615.0,91.0,387.0,87.0,46.0,122188.0,2418.0,997.0,125.0
1332,75701.0,0.0,100.0,1226.0,34.0,0.0,1361.0,505.0,48.0,262.0,615.0,91.0,387.0,87.0,46.0,122188.0,2418.0,997.0,125.0


In [68]:
srprec_census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1334 entries, 0 to 1333
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   srprec_orig               1334 non-null   float64
 1   subindex                  1334 non-null   float64
 2   pctsrprec_tract           1334 non-null   float64
 3   employed_wgt              1334 non-null   float64
 4   unemployed_wgt            1334 non-null   float64
 5   empl_military_wgt         1334 non-null   float64
 6   not_inlaborforce_wgt      1334 non-null   float64
 7   working_women_wgt         1334 non-null   float64
 8   parents_work_under6_wgt   1334 non-null   float64
 9   parents_work_0617_wgt     1334 non-null   float64
 10  occ_mgmt_sci_art_wgt      1334 non-null   float64
 11  occ_service_sector_wgt    1334 non-null   float64
 12  occ_sales_gen_office_wgt  1334 non-null   float64
 13  occ_constr_maintc_wgt     1334 non-null   float64
 14  occ_manu

#### Step 2:  Merge in Vote data on 'srprec'

In [ ]:
#may need to rename traceability column in advance of second merge
#should be dropped already by .groupby() aggregation step, but a reminder just in case:

# df.rename(columns={'_merge':'census_merge'}, inplace=True)

#### Vote dataset

In [69]:
vote18 = pd.read_pickle('./data/trend18.pkl')
vote18.shape

(1334, 26)

In [70]:
vote18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1334 entries, 0 to 1545
Data columns (total 26 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   county    1334 non-null   int64  
 1   srprec    1334 non-null   Int64  
 2   cddist    1334 non-null   int64  
 3   TOTREG    1334 non-null   int64  
 4   TOTVOTE   1334 non-null   int64  
 5   CNGDEM01  1334 non-null   int64  
 6   CNGREP01  1334 non-null   int64  
 7   election  1334 non-null   object 
 8   type      1334 non-null   object 
 9   totreg_r  1334 non-null   float64
 10  dem       1334 non-null   float64
 11  rep       1334 non-null   float64
 12  aip       1334 non-null   float64
 13  paf       1334 non-null   float64
 14  msc       1334 non-null   float64
 15  lib       1334 non-null   float64
 16  nlp       1334 non-null   float64
 17  grn       1334 non-null   float64
 18  ref       1334 non-null   float64
 19  dcl       1334 non-null   float64
 20  male      1334 non-null   floa

In [71]:
vote18['srprec'] = vote18['srprec'].astype('int64')

In [72]:
vote18.head(3)

,county,srprec,cddist,TOTREG,TOTVOTE,CNGDEM01,CNGREP01,election,type,totreg_r,...,nlp,grn,ref,dcl,male,female,hispdem,hisprep,hispdcl,hispoth
0,30,10316,47,1735,1278,630,592,g18,V,1238.0,...,0.0,2.0,0.0,265.0,582.0,656.0,55.0,43.0,30.0,6.0
1,30,10317,47,2079,1214,638,513,g18,V,1184.0,...,0.0,4.0,0.0,286.0,588.0,596.0,165.0,60.0,71.0,10.0
2,30,10319,47,1448,999,495,462,g18,V,945.0,...,0.0,2.0,2.0,256.0,443.0,502.0,68.0,37.0,49.0,5.0


In [74]:
# examine the magnitude of the various party-registrations to see which are the largest affiliations
vote18[['totreg_r', 'dem', 'rep', 'dcl', 'aip', 'paf', 'msc', 'lib', 'nlp', 'grn', 'ref']].describe()

,totreg_r,dem,rep,dcl,aip,paf,msc,lib,nlp,grn,ref
count,1334.000000,1334.000000,1334.000000,1334.000000,1334.000000,1334.000000,1334.000000,1334.000000,1334.000000,1334.000000,1334.000000
mean,799.916792,277.739880,306.970765,185.386807,18.483508,1.151424,1.087706,6.274363,0.173163,2.229385,0.419790
std,541.892084,194.984814,247.478110,130.806513,14.065092,1.405544,1.330224,5.319752,0.455852,2.217337,0.761468
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,358.000000,119.000000,99.000000,81.250000,7.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000
50%,748.000000,261.000000,263.500000,172.000000,17.000000,1.000000,1.000000,5.000000,0.000000,2.000000,0.000000
75%,1224.000000,407.000000,464.500000,278.000000,28.000000,2.000000,2.000000,10.000000,0.000000,3.000000,1.000000
max,3012.000000,1281.000000,1289.000000,828.000000,90.000000,8.000000,10.000000,42.000000,3.000000,14.000000,6.000000


In [81]:
#drop party affiliations that are small.  Keep:  dem, rep, dcl (decline-to-state, i.e., not affiliated)
#drop 'type'.  V indicates all voters (by mail and at polling place).  all columns contain "V", so this column provides no additional information

vote18.drop(columns=['aip', 'paf', 'msc', 'lib', 'nlp', 'grn', 'ref', 'type'], inplace=True)

KeyError: "['aip' 'paf' 'msc' 'lib' 'nlp' 'grn' 'ref' 'type'] not found in axis"

In [82]:
vote18.head(3)

,county,srprec,cddist,TOTREG,TOTVOTE,CNGDEM01,CNGREP01,election,totreg_r,dem,rep,dcl,male,female,hispdem,hisprep,hispdcl,hispoth
0,30,10316,47,1735,1278,630,592,g18,1238.0,423.0,517.0,265.0,582.0,656.0,55.0,43.0,30.0,6.0
1,30,10317,47,2079,1214,638,513,g18,1184.0,490.0,356.0,286.0,588.0,596.0,165.0,60.0,71.0,10.0
2,30,10319,47,1448,999,495,462,g18,945.0,316.0,335.0,256.0,443.0,502.0,68.0,37.0,49.0,5.0


#### Merge vote data into srprec_census df on 'srprec'

In [83]:
combo2 = pd.merge(left=srprec_census, right=vote18, how='outer', left_on=['srprec_orig'], right_on=['srprec'], indicator=True)
combo2.head().T

,0,1,2,3,4
srprec_orig,2001,2002,2008,2009,2011
subindex,3,0,0,3,0
pctsrprec_tract,100,100,100,100,100
employed_wgt,2647,2881,3948,3045,2991
unemployed_wgt,161,217,278,147,74
empl_military_wgt,0,9,0,0,0
not_inlaborforce_wgt,1372,1525,1438,2459,1401
working_women_wgt,1152,1030,1656,1447,1486
parents_work_under6_wgt,268,257,561,273,334
parents_work_0617_wgt,783,1082,677,717,492


In [79]:
combo2.shape

(1334, 38)

In [84]:
combo2['_merge'].value_counts()

both          1334
right_only       0
left_only        0
Name: _merge, dtype: int64

In [80]:
combo2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1334 entries, 0 to 1333
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   srprec_orig               1334 non-null   float64 
 1   subindex                  1334 non-null   float64 
 2   pctsrprec_tract           1334 non-null   float64 
 3   employed_wgt              1334 non-null   float64 
 4   unemployed_wgt            1334 non-null   float64 
 5   empl_military_wgt         1334 non-null   float64 
 6   not_inlaborforce_wgt      1334 non-null   float64 
 7   working_women_wgt         1334 non-null   float64 
 8   parents_work_under6_wgt   1334 non-null   float64 
 9   parents_work_0617_wgt     1334 non-null   float64 
 10  occ_mgmt_sci_art_wgt      1334 non-null   float64 
 11  occ_service_sector_wgt    1334 non-null   float64 
 12  occ_sales_gen_office_wgt  1334 non-null   float64 
 13  occ_constr_maintc_wgt     1334 non-null   float6

In [ ]:
combo2.to_pickle('./data/combined18_extended.pkl')